<a href="https://colab.research.google.com/github/ShutoAraki/RevisedSimplexMethod/blob/master/Revised_Simplex_Method_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Revised Simplex Method homework for MATH 422: Operations Research


## Revised Simplex Method


For the LPP:

min $z = c^Tx$

$Ax=b$

$x≥0$

1.   Initialization: 

>> Find an intial feasible basis $B$ (HOW? idk) (Son: I haven't taken any CS class so apologize if my answer doesn't make any sense! My guess is that the user has to input the constraint matrix $A$ (including their chosen feasible basic solution), the cost vector $C^T$ and the $RHS$ vector $b$)

>> Compute $B^{-1}, \beta = B^{-1}b, z = c_B^T \beta$


2.   Pricing:

>> Compute $\pi^T = c_B^TB^{-1}$ and $d_j = c_j - \pi^Ta_j$.

>> if $d_j ≥ 0, \forall j \in R$ ($R$ is nonbasic variables):
>>> return OPTIMAL

>> else:
>>> Let $q$ be such that $d_q < 0$. Compute $\alpha_q = B^{-1}a_q$.

3. Ratio test: 

>> Compute $I = \{i|1≤i≤m, \alpha_q^i>0\} $.

>> if $I = \emptyset$:
>>> return UNBOUNDED

>> else:
>>> Compute $\theta = min_{i\in I}(\frac{\beta_i}{\alpha_q^i})$ and $p$ such that $\theta = \frac{\beta_p}{\alpha_q^p}$

4. Update:

>> Update the $p$-th basic variable from $x_{k_p}$ to $x_q$.

>> Update the $p$-th component of $\beta$ corresponded from $x_{k_p}$ to $x_q$.

>> Replace the $p$-th column of the basis (which was $a_{k_p}$)

## Problems

### (i)
minimize: $z = 6x_1 + 3x_2 + 4x_3$

subject to: 

$x_1 + 6x_2 + x_3 = 10$

 $2x_1 + 3x_2 + x_3 = 15$
 
$x_1, x_2 ≥ 0$


### (ii)
maximize: $z = 10x_1 + 11x_2$

subject to:

$x_1 + 2x_2 ≤ 150$

$3x_1 + 4x_2 ≤ 200$

$6x_1 + x_2 ≤ 175$ 

$x_1, x_2 ≥ 0$

### (iii) 

minimize: $z = 3x_1 + 2x_2 + 4x_3 + 6x_4$

subject to: 

$x_1 + 2x_2 + x_3 + x_4 ≥ 100$

$2x_1 + x_2 + 3x_3 + 7x_4 ≥ 1500$

$x_1, x_2, x_3, x_4 ≥ 0$



## Code

What specific data structures should we use? Pandas DataFrames or Numpy arrays? I know Numpy has inverse matrix functions.

(Son: I'm using Mathematica or Mathlab since they're designed for mathematicians, but I think Numpy is a really good choice :) )


Also, should we take user inputs in main to run this algorithm?


In [0]:
# Import dependencies
import sys
import numpy as np
import itertools

In [0]:
def user_input():
  n = int(input("How many variables?: "))
  m = int(input("How many constraints?: "))
  
  A = []
  ops = []
  b = []
  
  # Objective function setup
  objType = input("Enter 'min' or 'max': ")
  if (objType.lower() != 'min' and objType.lower() != 'max'):
    print("You need to type 'min' or 'max'")
  
  c = list(map(float, input("Enter costs separated by a space: ").split()))
  assert len(c) == n, "The number of costs don't match with the number of variables %d" % n
  
  # Set up the m x n matrix A
  for i in range(m):
    print("======= Constraint {} =======".format(i+1))
    a = list(map(float, input("Enter coefficients separated by a space: ").split()))
    assert len(a) == n, "The coefficients don't match with the number of variables %d" % n
      
    op = int(input("Type 0 for ≤, 1 for =, or 2 for ≥: "))
    assert 0 <= op <= 2, "The constraint type has to be an integer in [0, 2]"
    ops.append(op)
      
    b.append(int(input("RHS?: ")))
    
    A.append(a)
    print()
    
  # Convert everything to numpy arrays
  A = np.array(A)
  b = np.array(b)
  
  # Add slack and surplus vars --> Standard form
  for i in range(m):
    # ≤ constraint
    if ops[i] == 0:
      # newCol is the column vector with one element being one
      newCol = np.zeros((m, 1))
      newCol[i] = 1
      # appending the newCol to the right of A
      A = np.hstack((A, newCol))
      # expand c as well
      c.append(0)
      n += 1
    # ≥ constraint
    elif ops[i] == 2:
      newCol = np.zeros((m, 1))
      # -1 here because it's a surplus variable
      newCol[i] = -1
      A = np.hstack((A, newCol))
      c.append(0)
      n += 1
      
  c = np.array(c)
  if (objType == 'max'):
    c = -c
    
  return np.array(A), b, c

In [0]:
def find_initial_feasible_soln(A, b):
  m, n = A.shape
  
  # bsv_idx: indices for basic variables
  indices = [i for i in range(n)]
  bsv_idx = itertools.combinations(indices, m)
  
  # All possible combinations of basic variables
  for bsv in bsv_idx:
    B = A[:, bsv]
    try:
      B_inv = np.linalg.inv(B)
      beta = np.matmul(B_inv, b)
      if (beta >= 0).all():
        return B, B_inv, beta, list(bsv), [x for x in indices if x not in list(bsv)]
    except:
      pass # B_inv does not exist
  
  print("Initial Feasible Solution could not be found.")
  sys.exit()

In [0]:
def revisedSimplexMethod():
  A, b, c = readInput('q3.txt')
  
  # Initialization
  B, B_inv, beta, bsv, nonbsv = find_initial_feasible_soln(A, b)
  
  print("Initial Feasible Solution:")
  print("\tBasic Variables: ", end="")
  for i in bsv:
    print("x{}".format(i+1), end=" ")
  print()
  print("\tNon Basic Variables: ", end="")
  for i in nonbsv:
    print("x{}".format(i+1), end=" ")
  print()
  print("\t" + str(beta))
  
  print("Initial Objective Value:")
  print("\tCost:", abs(c))
  z = abs(np.matmul(np.transpose(c[bsv]), beta))
  print("\t" + str(z))
  
  while(True):
    
    # Pricing
    pi_T = np.transpose(np.matmul(np.transpose(c[bsv]), B_inv))
    reduced_costs = []

    for nbv in nonbsv:
      reduced_costs.append(c[nbv] - np.matmul(pi_T, A[:, nbv]))

    reduced_costs = np.array(reduced_costs)

    print(reduced_costs)

    if (reduced_costs >= 0).all():
      print("OPTIMAL SOLUTION")
      print("\tBasic Variables: ", end="")
      for i in range(len(bsv)):
        print("x{} = {}".format(bsv[i]+1, beta[i]), end=" ")
      z = abs(np.matmul(np.transpose(c[bsv]), beta))
      print("\nObjective Value z* =", z)
      return

    # Ratio Test

    minidx = np.argmin(reduced_costs) # This is the working column index
    aq = A[:, minidx]
    alpha_q = np.matmul(B_inv, aq)
    ratio = []
  #   curr = 0

    if (alpha_q < 0).all():
      print("UNBOUNDED")
      return
    else:
      ratio = beta / alpha_q
  #     for x in alpha_q:
  #       ratio.append(beta[curr]/x)
  #       curr += 1

    entering_bv = nonbsv[minidx]
    idxmin2 = np.argmin(ratio)
    exiting_bv = bsv[np.argmin(ratio)]
    print(entering_bv)
    print(exiting_bv)

    # Update

    idx_bv = idxmin2
    idx_nbv = minidx
    print(idx_bv)
    print(idx_nbv)
    bsv[idx_bv] = entering_bv
    nonbsv[idx_nbv] = exiting_bv
    print(bsv)
    print(nonbsv)

    B = A[:, bsv]
    B_inv = np.linalg.inv(B)
    beta = np.matmul(B_inv, b)
  
  
    
  
  
  
  
  
  
  
  
  
revisedSimplexMethod()

Initial Feasible Solution:
	Basic Variables: x1 x5 
	Non Basic Variables: x2 x3 x4 x6 
	[750. 650.]
Initial Objective Value:
	Cost: [3. 2. 4. 6. 0. 0.]
	2250.0
[ 0.5 -0.5 -4.5  1.5]
3
0
0
2
[3, 4]
[1, 2, 0, 5]
[1.14285714 1.42857143 1.28571429 0.85714286]
OPTIMAL SOLUTION
	Basic Variables: x4 = 214.28571428571428 x5 = 114.28571428571428 
Objective Value z* = 1285.7142857142858


**Code**
I used Mathematica for problem ii and it works fine, but I don't know if this seems like a killing method for Linear Programming or not since Mathematica is powerfully designed with matrice operations.

In [0]:
In[1]= (*Choose problem ii as an example*)
(*Input the number of basic and nonbasic variables (x and y)*)
x = Input[]
y = Input[]

Out[2]= 3

Out[3]= 2

In[2]= (*Input the cost vector for all variables*)
c = Table[Input[], {i, 1}, {j, x + y}]
(*Create the cost vector of basic variables*)
cB = c[[1 ;; 1, y + 1 ;; x + y]]

Out[4]= {{10, 11, 0, 0, 0}}

Out[5]= {{0, 0, 0}}

In[3]= (*Input the whole constraint matrix A from left to right, \
above to below, with columns of the basic variables located at the \
end*) 
A = Table[Input[], {i, x}, {j, x + y}]
(*Create the starting basic matrix B*)
B = A[[1 ;; x, y + 1 ;; x + y]]
(*Create the starting nonbasic matrix AN*)
AN = A[[1 ;; x, 1 ;; y]]

Out[6]= {{1, 2, 1, 0, 0}, {3, 4, 0, 1, 0}, {6, 1, 0, 0, 1}}

Out[7]= {{1, 0, 0}, {0, 1, 0}, {0, 0, 1}}

Out[8]= {{1, 2}, {3, 4}, {6, 1}}

In[4]= (*Input the RHS vector*)
b = Table[Input[], {i, x}, {j, 1}]

Out[9]= {{150}, {200}, {175}}

SyntaxError: ignored

In [0]:
def readInput(fileName):
    f = open(fileName, 'r')
    line = f.readline().strip('\n').split(',')
    n = int(line[0].strip())
    m = int(line[1].strip())
    objType = line[2]
    c = f.readline().strip('\n').split(',')
    c = list(map(lambda x: float(x), c))
    A = []
    ops = []
    b = []
    for i in range(m):
        line = f.readline().strip('\n').split(',')
        line = list(map(lambda x: x.strip(), line))
        temp = list(map(float, line[0:n]))
        A.append(temp)
        ops.append(line[n])
        b.append(float(line[n+1]))

    f.close()

    # Adding surplus and slack variables

    for i in range(m):
      # ≤ constraint
      if ops[i] == '<':
        # newCol is the column vector with one element being one
        newCol = np.zeros((m, 1))
        newCol[i] = 1
        # appending the newCol to the right of A
        A = np.hstack((A, newCol))
        # expand c as well
        c.append(0)
        n += 1
      # ≥ constraints
      elif ops[i] == '>':
        newCol = np.zeros((m, 1))
        # -1 here because it's a surplus variable
        newCol[i] = -1
        A = np.hstack((A, newCol))
        c.append(0)
        n += 1

    c = np.array(c)
    if (objType == 'max'):
      c = -c

    # return int(n), int(m), objType, c, A, ops, b
    return A, b, c

In [0]:
In[5]= (*Iteration*)
While[
 Min[zc = cB.Inverse[B].A - c] < 0,(*Check the optimal condition*)
 
 zc = cB.Inverse[B].A - c;(*Print out the reduced costs*)
 Inverse[B];(*Print out the inverse matrix*)
 Inverse[B].b;(*Print out the RHS*)
 cB.Inverse[B].b; (*Print out the value of objective function*)
 
 p1 = Part[Position[Flatten[zc], Min[zc]], 1, 1];
    (*Find the most negative reduced cost and hence the entering \
variable to the current basic system*)
 
 d = Diagonal[
   Inverse[B].b.(1/Transpose[Inverse[B].AN[[All, p1 ;; p1]]])]; 
    (*Calculate the column ratios*)
 d' = Select[d, # > 0 &];(*Find the ratios that are positive*)
 p2 = Part[Position[d, Min[d']], 1, 1];
    (*Find the minimum positive ratio and hence the leaving \
variable of the current basic system*)
 
 AN[[All, p1]] = B[[All, p2]]; (*Update the nonbasic matrix*)
 B[[All, p2]] = A[[All, p1]]; (*Update the basic matrix*)
 cB[[All, p2]] = c[[All, p1]]; (*Update the cost vector of basic variables*)
 ]


Out[10]= {{0, 0, 0, 8/3, 1/3}}

Out[11]= {{1, -(11/21), 2/21}, {0, 2/7, -(1/7)}, {0, -(1/21), 4/21}}

Out[12]= {{1300/21}, {225/7}, {500/21}}

Out[13]= {{1775/3}}

Out[14]= 1

Out[15]= {650, -225, 125}

Out[16]= {650, 125}

Out[17]= 3

Out[18]= {1, 3, 6}

Out[19]= {1, 3, 6}

Out[20]= {10}